# Hypothesis 2: Proximity to Research Institutions and Patent Density
- We expect that counties with prominent research universities (e.g., Stanford, MIT, UIUC) consistently see higher patent activity. This hypothesis explores whether proximity to academic research hubs correlates with innovation output.
- From feedback, we heard it would be great to make function on distance caculation, but hypothesis 2 is more about gathered around counties rather than distance.
- Datasets used in hypothesis 2: g_location_disambiguated.tsv, g_inventor_disambiguated.tsv, Colleges_and_Universities_-3122497483864735259.csv, and 2025-Public-Data-File.xlsx
- We decided to approach in 3 ways.
1. Brief check through scatter map
2. County density map
3. Summary statistics of counties' patent application quantity by counties with university vs. without university

## Package import

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import json
import requests
from assist_scripts import merge, fips_merger

## Import files
- Patent application location dataset, "g_location_disambiguated.tsv":"https://s3.amazonaws.com/data.patentsview.org/download/g_location_disambiguated.tsv.zip"
- Patent inventor dataset, "g_inventor_disambiguated.tsv":"https://s3.amazonaws.com/data.patentsview.org/download/g_inventor_disambiguated.tsv.zip"
- List of universities with location information, "Colleges_and_Universities_-3122497483864735259.csv":"https://hifld-geoplatform.hub.arcgis.com/datasets/geoplatform::colleges-and-universities/about"
- List universities with research spending filter, "2025-Public-Data-File.xlsx":"https://carnegieclassifications.acenet.edu/wp-content/uploads/2025/04/2025-Public-Data-File.xlsx"

In [2]:
# dataset about location of patent application, includes location ID, latitude, and longitude those are necessary for the mapping
location_df = pd.read_csv("data/g_location_disambiguated.tsv", sep="\t", header=0,
                          usecols=['location_id', 'disambig_state', 'disambig_country', 'latitude', 'longitude', 'state_fips', 'county_fips'],
                          dtype={'location_id': str, 'disambig_state': str, 'disambig_country': str, 'latitude': float, 'longitude': float}
                          )

In [3]:
# dataset about inventor of technology that has patent application, which will be connected with location dataset for indicate each inventor's location
inventor_df = pd.read_csv("data/g_inventor_disambiguated.tsv", sep="\t", header=0,
                          usecols=['location_id'],
                          dtype={'location_id': str})

In [4]:
# dataset about university latitude and longitude
university_df = pd.read_csv("data/Colleges_and_Universities_-3122497483864735259.csv",
                            usecols=['IPEDSID', 'COUNTYFIPS', 'LATITUDE', 'LONGITUDE'],
                            dtype={'IPEDSID': str, 'COUNTYFIPS': str, 'LATITUDE': float, 'LONGITUDE': float})

In [5]:
# dataset about university that indicate research university
research_df = pd.read_excel("data/2025-Public-Data-File.xlsx", sheet_name="data",
                            usecols=['unitid', 'instnm', 'research2025'],
                            dtype={'unitid': str, 'instnm': str, 'research2025': int})

In [6]:
# Download U.S. counties GeoJSON (simplified)
geojson_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
counties_geo = requests.get(geojson_url).json()

## 1. Scatter map

### 1-1. Merging dataframes for combining required data for analysis

In [7]:
# The merge code from chatgpt, I asked how to add location data to the inventor table
# Merging inventor dataset with location dataset for getting latitude and longitude, and country of inventor
inv_loc_df = merge(merge_on_df=inventor_df, merge_from_df=location_df,
                  merge_from_keep=['location_id', 'latitude', 'longitude', 'state_fips', 'county_fips', 'disambig_country'],
          foreign_key_left="location_id")

In [8]:
# Filter out rows with universities with very high spending on research
research_df = research_df[research_df['research2025'] == 1]

# Merge with filtered university dataframe with location information (latitude and longitude) for plotting points on map
reseach_uni_df = merge(merge_on_df=research_df, merge_from_df=university_df,
                  merge_from_keep=["IPEDSID", 'COUNTYFIPS', 'LATITUDE', 'LONGITUDE'],
          foreign_key_left="unitid", foreign_key_right="IPEDSID")

# Dropping IPEDSID since this is same as unitid, dropping unecssary column
reseach_uni_df = reseach_uni_df.drop(columns=["IPEDSID"])

In [9]:
# Limit patent applicants, the inventors to US only
us_inv_loc_df = inv_loc_df[inv_loc_df['disambig_country'] == 'US']

# Then sample out by 500,000 as the dataset contains too much samples to visualized (lagged)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html/
loc_sample_df = us_inv_loc_df.sample(n=500000, random_state=42)

### 1-2. Visualization on scatter map
- This is visualization that blue dots are individual inventors with patents, whereas red starts are very high spending research universities on a U.S. territory map.
- loc_sample_df, the subsampled version of dataset used for performance issue.
- Got an assistence from ChatGPT for the plot

In [10]:
# The plotly codes are from ChatGPT
fig = go.Figure()

# 🟦 Add individuals
fig.add_trace(go.Scattergeo(
    lon = loc_sample_df['longitude'],
    lat = loc_sample_df['latitude'],
    hovertext = loc_sample_df['location_id'],  # or individual ID
    mode = 'markers',
    marker=dict(
        size=4,
        color='blue',
        opacity=0.6
    ),
    name='Patent'
))

# 🟥 Add universities
fig.add_trace(go.Scattergeo(
    lon = reseach_uni_df['LONGITUDE'],
    lat = reseach_uni_df['LATITUDE'],
    hovertext = reseach_uni_df['instnm'],
    mode = 'markers',
    marker=dict(
        size=7,
        color='red',
        symbol='star'
    ),
    textposition='top center',
    name='Universities'
))

# 🌎 Map layout
fig.update_layout(
    title='Individuals and Universities on USA Map',
    geo=dict(
        scope='usa',
        showland=True,
        landcolor='lightgray',
        showlakes=True,
        lakecolor='lightblue',
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# uncomment out below if you want to show in a separate browser or window
#pio.renderers.default = 'browser'
fig.show()

#### Result for scatter map
- In a brief sense, it looks like patent applicants (inventors) likely gathered around area nearby research universities.
- Still not clear enough to makes the conclusion, so proceed to the next visualization

## County density map
- This is a density heatmap based on number of patent application by inventors with red starts as universities
- Helped from ChatGPT, for the visualization part

In [11]:
# Also got helped form ChatGPT for the density visualization
## Drop rows with missing FIPS values
clean_df = inv_loc_df.dropna(subset=['state_fips', 'county_fips'])

# Convert to integers (handle float like 17.0 → 17)
# Combine to full 5-digit county FIPS code (e.g., 17031 for Cook County, IL)
clean_df['county_fips'] = fips_merger(clean_df['state_fips'], clean_df['county_fips'])

# Count individuals per county
county_counts = clean_df['county_fips'].value_counts().reset_index()
county_counts.columns = ['county_fips', 'count']

/var/folders/2j/sh023cfx5kd0tgwfn0nptbrc0000gn/T/ipykernel_12571/1829273765.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
# Also got helped form ChatGPT for the density visualization
# Choropleth for individual density
fig = px.choropleth(
    county_counts,
    geojson=counties_geo,
    locations='county_fips',
    color='count',
    color_continuous_scale="Blues",
   # range_color=(0, 12),
    scope="usa",
    labels={'count': 'Patents'},
    title="Patent Density per County with Universities"
)

# Add universities as points
fig.add_trace(go.Scattergeo(
    lon = reseach_uni_df['LONGITUDE'],
    lat = reseach_uni_df['LATITUDE'],
    hovertext = reseach_uni_df['instnm'],
    mode = 'markers',
    marker=dict(
        size=7,
        color='red',
        symbol='star'
    ),
    name='Universities'
))

# Update layout for map styling
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

#pio.renderers.default = 'browser'
fig.show()

- According to the visualizations, it is hard to compare counties with and without the universities

## Summary statistics
- Calculated summary statistics for counties' patent application amount with university vs. without university
- Helped from ChatGPT

In [13]:
# 1. mean density of patent application count for university counties
# - Add (total patent count for the county / total patent applications) for university counties
# - Divided by number of counties occupied with universities
# 2. mean density of patent application count for other than university counties

univ_counties = set(reseach_uni_df['COUNTYFIPS'])

# Flag whether each county has a university
county_counts['has_university'] = county_counts['county_fips'].isin(univ_counties)

# Compare average density
summary = county_counts.groupby('has_university')['count'].describe()
print(summary)

                 count          mean           std    min     25%      50%  \
has_university                                                               
False           3022.0   1623.593977   7647.321405    1.0    31.0    124.0   
True             153.0  41576.032680  98580.299467  530.0  4368.0  13235.0   

                    75%       max  
has_university                     
False             544.5  213359.0  
True            35304.0  962585.0  


- The summary statistic results show that high mean, but unreliable due to huge gap between min and max patent application amount.
- So, by checking minimum, median, maximum, we can conclude counties with universities have more patent applicants
- This is comparable result as counties with universities are small (153 vs 3022) but showed higher minimum, median, and maximum patent applicants.
- Therefore, the hypothesis 2, counties with prominent research universities (e.g., Stanford, MIT, UIUC) consistently see higher patent activity, is likely right according to minimum, median, and maximum amount of patent applicants are more around universities.